In [237]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import itertools
import os

In [193]:
#getting a sample of the page 
url = 'http://www.fagen.ufu.br/equipe'
req = requests.get(url)
soup = bs(req.text, 'html.parser')


#determining the last page to help me on the loop
pagestring = soup.find('a', title='Ir para a última página').get('href')
lastpage = int(pagestring[pagestring.find('=')+1:])
lastpage

9

In [180]:
people[0].select_one('a[href*=lattes]').string

'http://lattes.cnpq.br/4897620765386940'

In [212]:
def people_data(people):
    peoplelist = []
    for person in people:
        #person name
        if person.a is None:
            name = None
        else:
            name = person.a.string 
        
         #control variable
        print(name)
            
        #lattes
        if person.select_one('a[href*=lattes]') is None: 
            lattes = None
        else:
            lattes = person.select_one('a[href*=lattes]')['href']

        #research field
        if person.p is None:
            researchfield = None
        else:
            researchfield = person.p.string 

        #email
        if person.select_one('a[href*=mailto]') is None:
            email = None
        else:
            email = person.select_one('a[href*=mailto]').string

        #admission date
        if person.find('span', class_='date-display-single') is None:
            admissiondate = None
        else:
            admissiondate = person.find('span', class_='date-display-single').text 

        peoplelist.append([name, lattes, researchfield, email, admissiondate])    
    return peoplelist

peoplelist = []

for page in range(0,lastpage):
    print('---- Starting page ' + str(page) + '----')
    req = requests.get(url + '?page=' + str(page))
    soup = bs(req.text, 'html.parser')
    people = soup.find_all('div', class_='col-sm-9')
    peoplelist = peoplelist + people_data(people)
    print('---- End of page ' + str(page) + '----\n')

---- Starting page 0----
Aleandra da Silva Figueira Sampaio
Ana Beatriz Pereira Reis
André Francisco Alcântara Fagundes
Andrea Costa Van Herk Vasconcelos
Antonio Sérgio Torres Penedo
Aracy Alves de Araújo
Camila de Araújo
Carla Bonato Marcolin
---- End of page 0----

---- Starting page 1----
Carlos Henrique Viola
Carlos Roberto Domingues
Catarine Palmieri Pitangui Tizziotti
Cíntia Rodrigues de Oliveira
Cláudio Luiz Miotto
Cristiane Betanho
Cristiano Augusto Borges Forti
Cristiano Henrique Antonelli da Veiga
---- End of page 1----

---- Starting page 2----
Cristina Damm Forattini Dias
Darly Fernando Andrade
Diego Batista de Carvalho
Edileusa Godói de Sousa
Eduardo Giarola
Élcio Eduardo de Paula Santana
Etienne Cardoso Abdala
Eunice Henriques Pereira Vilela
---- End of page 2----

---- Starting page 3----
Eustáquio São José de Faria
Fabiolla Valeria Gonçalves
Fernanda Francielle de Oliveira Malaquias
Fernanda Maciel Peixoto
Flávio Luiz de Moraes Barboza
Henrique Geraldo Rodrigues
Jacquel

In [266]:
#insert into a dataframe
people_df = pd.DataFrame(peoplelist, columns = ('Name', 'Lattes', 'Field of Interest','Email', 'Admission Date'))
people_df

,Name,Lattes,Field of Interest,Email,Admission Date
0,Aleandra da Silva Figueira Sampaio,http://lattes.cnpq.br/4897620765386940,"Tecnologia Educacional, Produtos de Software E...",aleandra@ufu.br,11/02/2011
1,Ana Beatriz Pereira Reis,None,None,ana.reis@ufu.br,24/08/2021
2,André Francisco Alcântara Fagundes,http://lattes.cnpq.br/4605993276758331,"Comportamento do consumidor, Marketing esporti...",andre.fagundes@ufu.br,31/07/2008
3,Andrea Costa Van Herk Vasconcelos,http://lattes.cnpq.br/5429960737925655,"Empreendedorismo, Organizações, Gestão de Pess...",andreaherk@ufu.br,04/02/2014
4,Antonio Sérgio Torres Penedo,http://lattes.cnpq.br/6987375118761399,"Mercado de Capitais, Análise de Investimentos,...",penedo@ufu.br,20/11/2012
...,...,...,...,...,...
67,Rodrigo Fernandes Malaquias,http://lattes.cnpq.br/9135874740234258,None,rodrigofmalaquias@ufu.br,23/01/2009
68,Rodrigo Miranda,http://lattes.cnpq.br/8287221667041691,"Organizações, Estratégia, Terceiro Setor, Cult...",rodrigomiranda@ufu.br,24/07/2009
69,Sany Karla Machado,http://lattes.cnpq.br/7164684823979819,Sociologia e Antropologia do Consumo.,sany@ufu.br,06/07/2010
70,Sérgio Luiz do Amaral Moretti,http://lattes.cnpq.br/1686957518540720,None,sergiomoretti@ufu.br,14/08/2019


In [248]:
#export to xlsx 
if not os.path.exists('data'):
    os.mkdir('data')
people_df.to_excel('~/OneDrive/Faculdade/Orientadores/Busca orientador/docentes-FAGEN-UFU/data/orientadoresfagen.xlsx')